In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
73,ABW,Aruba,2020-06-04,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
220,AFG,Afghanistan,2020-06-04,17267,758,294,24,443.559,19.472,7.552,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
295,AGO,Angola,2020-06-04,86,0,4,0,2.617,0.000,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-03,France,151487.0,0.0,28995.0,0.0,0.0,0.0
0,2020-06-03,China,84221.0,0.0,4660.0,0.0,0.0,0.0
0,2020-06-03,Italy,233829.0,0.0,33610.0,0.0,0.0,0.0
0,2020-06-03,Spain,240194.0,0.0,28512.0,0.0,0.0,0.0
0,2020-06-03,United States,1855181.0,0.0,107106.0,0.0,0.0,0.0
0,2020-06-03,World,6478838.0,0.0,385101.0,0.0,0.0,0.0
0,2020-06-03,United Kingdom,279943.0,0.0,39923.0,0.0,0.0,0.0
0,2020-06-03,Germany,182674.0,0.0,8563.0,0.0,0.0,0.0
0,2020-06-03,Iran,160452.0,0.0,8025.0,0.0,0.0,0.0
0,2020-06-03,Turkey,166350.0,0.0,4606.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-04,France,151487.0,151677.0,28995.0,29021.0,-190.0,-26.0
0,2020-06-04,China,84221.0,84160.0,4660.0,4638.0,61.0,22.0
0,2020-06-04,Italy,233829.0,233836.0,33610.0,33601.0,-7.0,9.0
0,2020-06-03,Spain,240194.0,240326.0,28512.0,27940.0,-132.0,572.0
0,2020-06-04,United States,1855181.0,1851520.0,107106.0,107175.0,3661.0,-69.0
0,2020-06-04,World,6478838.0,6475644.0,385101.0,386544.0,3194.0,-1443.0
0,2020-06-04,United Kingdom,279943.0,279856.0,39923.0,39728.0,87.0,195.0
0,2020-06-04,Germany,182674.0,182764.0,8563.0,8581.0,-90.0,-18.0
0,2020-06-04,Iran,160452.0,160696.0,8025.0,8012.0,-244.0,13.0
0,2020-06-04,Turkey,166350.0,166422.0,4606.0,4609.0,-72.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")